In [ ]:
!pip install transformers

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

**Mounting Drive**

In [ ]:
from google.colab import drive
from google.colab import files
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


**REBEL MODEL**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
}



Exception ignored in: <function _xla_gc_callback at 0x7eb3ad3c4280>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 

**KNOWLEDGE GRAPH GENERATION**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from IPython.display import display, Javascript

#text = "Total contract values are expected to be little changed on a sequential basis. Supply-side challenges remain, but the demand outlook remains strong"
news_folder='/content/drive/MyDrive/Major_Project/News/'
triplets_dataset=[]
cnt=0
for comp in os.listdir(news_folder):
    print(comp)
    cnt=cnt+1
    news_df =pd.read_csv(news_folder+comp)
    triplets_list = []
    for i in range(len(news_df)) :
        #text = "The government along with LIC is selling nearly 61 per cent stake in IDBI Bank and had in October 2022, invited bids from buyers"
        text=news_df['headline'][i]
        # Tokenizer text
        model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')

        # Generate
        generated_tokens = model.generate(
            model_inputs["input_ids"].to(model.device),
            attention_mask=model_inputs["attention_mask"].to(model.device),
            **gen_kwargs,
        )

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

        # Extract text


        # Extract triplets
        for idx, sentence in enumerate(decoded_preds):
        #    print(f'Prediction triplets sentence {idx}')
            triplet=extract_triplets(sentence)
        #   print(triplet)
            triplets_list.extend(triplet)

    trip_df=pd.DataFrame(triplets_list,columns=['head','type','tail'])
    trip_df.to_csv(comp[:-4]+'_triplets.csv')
    # files.download(comp[:-4]+'_triplets.csv')
    # time.sleep(5)
    # while any(fname.startswith(comp[:-4]) for fname in os.listdir('/content')):
    #       time.sleep(5)
    triplets_dataset.append(triplets_list)


G = nx.Graph()
for triplet in triplets_list:
    subject, relation, obj = triplet
    print(triplet)
    G.add_node(triplet[subject])
    G.add_node(triplet[obj])
    G.add_edge(triplet[subject], triplet[obj], label=triplet[relation])

    # Visualization
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, font_weight='bold', node_size=2500, node_color='skyblue', font_size=8, edge_color='gray', width=3, alpha=0.7, font_color='black')
labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels,font_size=6)
plt.show()



**EMBEDDINGS GENERATION**

In [ ]:
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from sklearn.decomposition import PCA
import os

# Assuming your dataframe has columns 'subject', 'predicate', 'object'
triplet_folder='/content/drive/MyDrive/Major_Project/Triplets/'
dist_ent_emb=[]
dist_rel_emb=[]
trans_ent_emb=[]
trans_rel_emb=[]
count=0
for comp in os.listdir(triplet_folder):
    print(count,comp)
    count=count+1
    triplet_df=pd.read_csv(triplet_folder+comp)
    triples_factory = TriplesFactory.from_labeled_triples(triplet_df[['head', 'type', 'tail']].values,)

    training = triples_factory
    validation = triples_factory
    testing = triples_factory

    d=training
    id_to_entity={v: k for k, v in d.entity_to_id.items()}
    id_to_relation={v: k for k, v in d.relation_to_id.items()}

    for i in d.map_triples(d.triples):
        s,p,o =int(i[0]), int(i[1]), int(i[2])
        h,r,t = id_to_entity[s], id_to_relation[p], id_to_entity[o]

    result1 = pipeline(
        model='DistMult',
        loss="softplus",
        training=training,
        testing=testing,
        validation=validation,
        model_kwargs=dict(embedding_dim=50, random_seed=42),  # Increase the embedding dimension
        optimizer_kwargs=dict(lr=0.1),  # Adjust the learning rate
        training_kwargs=dict(num_epochs=25, use_tqdm_batch=False),)  # Increase the number of epochs)

    result2 = pipeline(
        model='TransE',
        loss="softplus",
        training=training,
        testing=testing,
        validation=validation,
        model_kwargs=dict(embedding_dim=50, random_seed=42),  # Increase the embedding dimension
        optimizer_kwargs=dict(lr=0.1),  # Adjust the learning rate
        training_kwargs=dict(num_epochs=25, use_tqdm_batch=False),  # Increase the number of epochs
    )

    # The trained model is stored in the pipeline result
    model1 = result1.model
    model2 = result2.model

    pca = PCA(n_components=2)
    entity_embeddings1 = model1.entity_representations[0](indices=None).detach().cpu().numpy()
    m = pca.fit(entity_embeddings1)
    eu = m.transform(entity_embeddings1)
    relation_embeddings1 = model1.relation_representations[0](indices=None).detach().cpu().numpy()
    ru = pca.transform(relation_embeddings1)

    pca = PCA(n_components=2)
    entity_embeddings2 = model2.entity_representations[0](indices=None).detach().cpu().numpy()
    m = pca.fit(entity_embeddings2)
    eu = m.transform(entity_embeddings2)
    relation_embeddings2 = model2.relation_representations[0](indices=None).detach().cpu().numpy()
    ru = pca.transform(relation_embeddings2)

    dist_ent_emb.append(entity_embeddings1)
    trans_ent_emb.append(entity_embeddings2)
    dist_rel_emb.append(relation_embeddings1)
    trans_rel_emb.append(relation_embeddings2)






# Display the first few triples

ModuleNotFoundError: No module named 'pykeen'

In [ ]:
target=[1,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1,1,0,1,1,0,0,0,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,0,1,1,1,0,1,0,1,1,1,1,0,1,0,0,0,1,1,1,0,1,1,0,0,1,1,0,1,0,0,1,1,1,1,0,0,0]

In [ ]:
len(target)

102

In [ ]:
import numpy as np

In [ ]:
y_train=np.array(target)

In [ ]:
dr=dist_rel_emb.copy()
de=dist_ent_emb.copy()
tr=trans_rel_emb.copy()
te=trans_ent_emb.copy()
# dist_rel_emb=dr.copy()
# dist_ent_emb=de.copy()
# trans_rel_emb=tr.copy()
# trans_ent_emb=te.copy()

In [ ]:
size=50
ln=0
for i in range(len(de)):
  ln=len(de[i])
  for j in range(ln,size):
    de[i]=np.concatenate((de[i],[de[i][j%ln]]),axis=0)
for i in range(len(dr)):
  ln=len(dr[i])
  for j in range(ln,size):
    dr[i]=np.concatenate((dr[i],[dr[i][j%ln]]),axis=0)
for i in range(len(tr)):
  ln=len(tr[i])
  for j in range(ln,size):
    tr[i]=np.concatenate((tr[i],[tr[i][j%ln]]),axis=0)
for i in range(len(te)):
  ln=len(te[i])
  for j in range(ln,size):
    te[i]=np.concatenate((te[i],[te[i][j%ln]]),axis=0)


In [ ]:
for i in range(len(dist_rel_emb)):
  dist_rel_emb[i] = dist_rel_emb[i][:7]
for i in range(len(trans_ent_emb)):
  trans_ent_emb[i] = trans_ent_emb[i][:7]
for i in range(len(dist_ent_emb)):
  dist_ent_emb[i] = dist_ent_emb[i][:7]
for i in range(len(trans_rel_emb)):
  trans_rel_emb[i] = trans_rel_emb[i][:7]

In [ ]:
type(dist_rel_emb)

list

In [ ]:
tr[0].shape

(50, 50)

In [ ]:
dist_rel_emb[1].shape

(24, 50)

In [ ]:
import numpy as np
dist_merged  = []
trans_merged = []
for i in range(len(dist_ent_emb)):
  dist_merged.append(np.concatenate((dr[i], de[i])))
  trans_merged.append(np.concatenate((tr[i], te[i])))

In [ ]:
print(dist_merged[i].shape)
print(trans_merged[i].shape)

(100, 50)
(100, 50)


In [ ]:
print(trans_merged[0].shape)
print(dist_merged[0].shape)

(14, 50)
(14, 50)


In [ ]:
X1_train = np.array(dist_merged)
X2_train = np.array(trans_merged)

In [ ]:
X1_train.shape

(102, 14, 50)

In [ ]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, y1_train, y1_test = train_test_split(X1_train, y_train, test_size=0.2, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print(X1_train.shape)
print(X2_train.shape)

(81, 14, 50)
(81, 14, 50)


**LSTM MODEL**

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define your model
model = Sequential()

# Add LSTM layer
model.add(LSTM(units=64, input_shape=(100,50)))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=64, activation='relu'))
# Add Dense layer for binary classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33,201
Trainable params: 33,201
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model1 = Sequential()

# Add LSTM layer
model1.add(LSTM(units=64, input_shape=(100,50)))
model1.add(Dense(units=16, activation='relu'))
model1.add(Dense(units=32, activation='relu'))
model1.add(Dense(units=64, activation='relu'))
# Add Dense layer for binary classification
model1.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 dense_4 (Dense)             (None, 16)                1040      
                                                                 
 dense_5 (Dense)             (None, 32)                544       
                                                                 
 dense_6 (Dense)             (None, 64)                2112      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33,201
Trainable params: 33,201
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X1_train, y1_train, batch_size=32, epochs=30)

Epoch 1/30
3/3 [==============================] - 4s 58ms/step - loss: 0.6924 - accuracy: 0.5432
Epoch 2/30
3/3 [==============================] - 0s 51ms/step - loss: 0.6890 - accuracy: 0.5432
Epoch 3/30
3/3 [==============================] - 0s 53ms/step - loss: 0.6860 - accuracy: 0.5432
Epoch 4/30
3/3 [==============================] - 0s 51ms/step - loss: 0.6832 - accuracy: 0.5432
Epoch 5/30
3/3 [==============================] - 0s 52ms/step - loss: 0.6795 - accuracy: 0.5432
Epoch 6/30
3/3 [==============================] - 0s 65ms/step - loss: 0.6759 - accuracy: 0.5432
Epoch 7/30
3/3 [==============================] - 0s 52ms/step - loss: 0.6706 - accuracy: 0.5432
Epoch 8/30
3/3 [==============================] - 0s 51ms/step - loss: 0.6652 - accuracy: 0.5432
Epoch 9/30
3/3 [==============================] - 0s 52ms/step - loss: 0.6570 - accuracy: 0.5432
Epoch 10/30
3/3 [==============================] - 0s 52ms/step - loss: 0.6480 - accuracy: 0.5432
Epoch 11/30
3/3 [============

In [ ]:
history1 = model1.fit(X2_train, y2_train, batch_size=32, epochs=30)

Epoch 1/30
3/3 [==============================] - 4s 95ms/step - loss: 0.6950 - accuracy: 0.4444
Epoch 2/30
3/3 [==============================] - 0s 91ms/step - loss: 0.6899 - accuracy: 0.6667
Epoch 3/30
3/3 [==============================] - 0s 88ms/step - loss: 0.6866 - accuracy: 0.7160
Epoch 4/30
3/3 [==============================] - 0s 91ms/step - loss: 0.6821 - accuracy: 0.7778
Epoch 5/30
3/3 [==============================] - 0s 52ms/step - loss: 0.6766 - accuracy: 0.7901
Epoch 6/30
3/3 [==============================] - 0s 56ms/step - loss: 0.6688 - accuracy: 0.8148
Epoch 7/30
3/3 [==============================] - 0s 61ms/step - loss: 0.6581 - accuracy: 0.8395
Epoch 8/30
3/3 [==============================] - 0s 56ms/step - loss: 0.6436 - accuracy: 0.8395
Epoch 9/30
3/3 [==============================] - 0s 51ms/step - loss: 0.6215 - accuracy: 0.8519
Epoch 10/30
3/3 [==============================] - 0s 51ms/step - loss: 0.5864 - accuracy: 0.8519
Epoch 11/30
3/3 [============

**PERFORMANCE METRICS OF DISTMULT EMBEDDINGS + LSTM**

In [ ]:
# Assuming X1_test and y1_test are your test data and labels respectively
# Predict on the test data
y_pred_prob = model.predict(X1_test)

# Convert probabilities to class labels (0 or 1)
y_pred = (y_pred_prob > 0.5).astype(int)

# Evaluate the model using sklearn.metrics.accuracy_score
from sklearn import metrics

# Calculate accuracy
accuracy = metrics.accuracy_score(y1_test, y_pred)*100
print("Accuracy :",accuracy)
print("Sensitivity : ", metrics.recall_score(y1_test , y_pred)*100,"%")
print("Specificity : ", metrics.recall_score(np.logical_not(y1_test) ,np.logical_not(y_pred))*100,"%")
print("Classification Report : ")
print(metrics.classification_report(y1_test, y_pred))

# Print the accuracy


1/1 [==============================] - 1s 500ms/step
Accuracy : 57.14285714285714
Sensitivity :  30.76923076923077 %
Specificity :  100.0 %
Classification Report : 
              precision    recall  f1-score   support

           0       0.47      1.00      0.64         8
           1       1.00      0.31      0.47        13

    accuracy                           0.57        21
   macro avg       0.74      0.65      0.56        21
weighted avg       0.80      0.57      0.54        21



**PERFORMANCE METRICS OF TRANS-E EMBEDDINGS + LSTM**

In [ ]:
y_pred_prob1 = model1.predict(X2_test)

# Convert probabilities to class labels (0 or 1)
y_pred1 = (y_pred_prob1 > 0.5).astype(int)

# Evaluate the model using sklearn.metrics.accuracy_score
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = metrics.accuracy_score(y2_test, y_pred)*100
print("Accuracy :",accuracy)
print("Sensitivity : ", metrics.recall_score(y2_test , y_pred)*100,"%")
print("Specificity : ", metrics.recall_score(np.logical_not(y2_test) ,np.logical_not(y_pred))*100,"%")
print("Classification Report : ")
print(metrics.classification_report(y2_test, y_pred))

1/1 [==============================] - 0s 489ms/step
Accuracy : 57.14285714285714
Sensitivity :  30.76923076923077 %
Specificity :  100.0 %
Classification Report : 
              precision    recall  f1-score   support

           0       0.47      1.00      0.64         8
           1       1.00      0.31      0.47        13

    accuracy                           0.57        21
   macro avg       0.74      0.65      0.56        21
weighted avg       0.80      0.57      0.54        21

